In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [29]:
url_inicial = 'https://revistamexicanadeurologia.org.mx/index.php/rmu/issue/archive'
url_root = 'https://revistamexicanadeurologia.org.mx/index.php/rmu/issue/archive'
r=requests.get(url_inicial)

In [30]:
soup = BeautifulSoup(r.text, 'html.parser')

In [31]:
box = soup.find('ul', class_='issues_archive')
volumen=box.findAll('div', class_='obj_issue_summary')
vol = [x.find('a').get('href')for x in volumen]
vol=[urljoin(url_root,i) for i in vol]
vol

['https://revistamexicanadeurologia.org.mx/index.php/rmu/issue/view/52',
 'https://revistamexicanadeurologia.org.mx/index.php/rmu/issue/view/51',
 'https://revistamexicanadeurologia.org.mx/index.php/rmu/issue/view/50',
 'https://revistamexicanadeurologia.org.mx/index.php/rmu/issue/view/49',
 'https://revistamexicanadeurologia.org.mx/index.php/rmu/issue/view/48',
 'https://revistamexicanadeurologia.org.mx/index.php/rmu/issue/view/47',
 'https://revistamexicanadeurologia.org.mx/index.php/rmu/issue/view/46',
 'https://revistamexicanadeurologia.org.mx/index.php/rmu/issue/view/45',
 'https://revistamexicanadeurologia.org.mx/index.php/rmu/issue/view/44',
 'https://revistamexicanadeurologia.org.mx/index.php/rmu/issue/view/43',
 'https://revistamexicanadeurologia.org.mx/index.php/rmu/issue/view/42',
 'https://revistamexicanadeurologia.org.mx/index.php/rmu/issue/view/41',
 'https://revistamexicanadeurologia.org.mx/index.php/rmu/issue/view/40',
 'https://revistamexicanadeurologia.org.mx/index.ph

In [32]:
vol2 =[]
for i in vol: 
    url_inicial1=i 
    r1 = requests.get(url_inicial1)
    soup1 = BeautifulSoup(r1.text, 'html.parser')
    box1 = soup1.find('div', class_='sections') 
    volumen1=box1.findAll('div', class_='obj_article_summary')
    vol1 = [x.find('a').get('href')for x in volumen1]
    vol2+=vol1

In [33]:
vol2

['https://revistamexicanadeurologia.org.mx/index.php/rmu/article/view/896',
 'https://revistamexicanadeurologia.org.mx/index.php/rmu/article/view/876',
 'https://revistamexicanadeurologia.org.mx/index.php/rmu/article/view/869',
 'https://revistamexicanadeurologia.org.mx/index.php/rmu/article/view/891',
 'https://revistamexicanadeurologia.org.mx/index.php/rmu/article/view/816',
 'https://revistamexicanadeurologia.org.mx/index.php/rmu/article/view/832',
 'https://revistamexicanadeurologia.org.mx/index.php/rmu/article/view/743',
 'https://revistamexicanadeurologia.org.mx/index.php/rmu/article/view/810',
 'https://revistamexicanadeurologia.org.mx/index.php/rmu/article/view/858',
 'https://revistamexicanadeurologia.org.mx/index.php/rmu/article/view/900',
 'https://revistamexicanadeurologia.org.mx/index.php/rmu/article/view/903',
 'https://revistamexicanadeurologia.org.mx/index.php/rmu/article/view/763',
 'https://revistamexicanadeurologia.org.mx/index.php/rmu/article/view/829',
 'https://re

In [34]:
def get_url_items(sopa,url):
    box1 = soup1.find('ul', class_='cmp_article_list articles') 
    volumen1=box1.findAll('div', class_='obj_article_summary')
    vol1 = [x.find('a').get('href')for x in volumen1]
    return vol2

In [35]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break

Estoy en la pagina https://revistamexicanadeurologia.org.mx/index.php/rmu/issue/archive


In [36]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

243

In [37]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [38]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Revista Mexicana de Urología'
    a='III. Medicina y Ciencias de la Salud'
    tem='Urología'
    s_item=BeautifulSoup(r.text,'html.parser')
    #titulo de revista
    try:
        titulo=tire
        content_book['Titulo Revista']=tire
    except AttributeError:
        content_book['Titulo Revista']=None
    #area
    try:
        area=a
        content_book['Area']=a
    except AttributeError:
        content_book['Area']=None
    #tematica
    try:
        tema=tem
        content_book['Tematica']=tem
    except AttributeError:
        content_book['Tematica']=None
    #titulo articulo
    try:
        titu=s_item.find('h1', class_='page_title').get_text(strip=True)
        content_book['Titulo Articulo']=titu
    except AttributeError:
        content_book['Titulo Articulo']=None
    #resumen
    try:
        resu=s_item.find('section', class_='item abstract').get_text(strip=True)
        content_book['Resumen']=resu.replace("Resumen","")
    except AttributeError:
        content_book['Resumen']=None
    #abstract
    try:
        abst=s_item.find('div', id='item cover_image').get_text(strip=True)
        content_book['Abstract']=abst
    except AttributeError:
        content_book['Abstract']=None
    #Link incial
    try:
        linkart=url
        content_book['Link Articulo']=linkart
    except AttributeError:
        content_book['Link Articulo']=None
    #link articulo
    try:
        link=s_item.find('a', class_='obj_galley_link pdf').get('href')
        content_book['Link PDF']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link PDF']=None
    return content_book

In [39]:
list_scraper=list_scraper[0:243]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2
estas escrapeando la pag 3
estas escrapeando la pag 4
estas escrapeando la pag 5
estas escrapeando la pag 6
estas escrapeando la pag 7
estas escrapeando la pag 8
estas escrapeando la pag 9
estas escrapeando la pag 10
estas escrapeando la pag 11
estas escrapeando la pag 12
estas escrapeando la pag 13
estas escrapeando la pag 14
estas escrapeando la pag 15
estas escrapeando la pag 16
estas escrapeando la pag 17
estas escrapeando la pag 18
estas escrapeando la pag 19
estas escrapeando la pag 20
estas escrapeando la pag 21
estas escrapeando la pag 22
estas escrapeando la pag 23
estas escrapeando la pag 24
estas escrapeando la pag 25
estas escrapeando la pag 26
estas escrapeando la pag 27
estas escrapeando la pag 28
estas escrapeando la pag 29
estas escrapeando la pag 30
estas escrapeando la pag 31
estas escrapeando la pag 32
estas escrapeando la pag 33
estas escrapeando la pag 34
estas escrapeando la pag 35
es

In [40]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo Revista,Area,Tematica,Titulo Articulo,Resumen,Abstract,Link Articulo,Link PDF
0,Revista Mexicana de Urología,III. Medicina y Ciencias de la Salud,Urología,Desarrollo y perspectivas de las mujeres itali...,None,None,https://revistamexicanadeurologia.org.mx/index...,https://revistamexicanadeurologia.org.mx/index...
1,Revista Mexicana de Urología,III. Medicina y Ciencias de la Salud,Urología,Rol de la laparoscopía en el tratamiento de la...,Objetivos:Conocer cuál es el rol del abordaje ...,None,https://revistamexicanadeurologia.org.mx/index...,https://revistamexicanadeurologia.org.mx/index...
2,Revista Mexicana de Urología,III. Medicina y Ciencias de la Salud,Urología,Evaluación del rendimiento de la Tabla de Part...,Antecedentes: Las tablas de Partin realizadas ...,None,https://revistamexicanadeurologia.org.mx/index...,https://revistamexicanadeurologia.org.mx/index...
3,Revista Mexicana de Urología,III. Medicina y Ciencias de la Salud,Urología,Rol de la nefrectomía citorreductora en Latino...,Introducción:El cáncer de riñón metastásico co...,None,https://revistamexicanadeurologia.org.mx/index...,https://revistamexicanadeurologia.org.mx/index...
4,Revista Mexicana de Urología,III. Medicina y Ciencias de la Salud,Urología,Enucleación laparoscópica de masas renales,Introducción: La enucleación tumoral (ET) de l...,None,https://revistamexicanadeurologia.org.mx/index...,https://revistamexicanadeurologia.org.mx/index...
...,...,...,...,...,...,...,...,...
238,Revista Mexicana de Urología,III. Medicina y Ciencias de la Salud,Urología,Ganglioneuroma suprarrenal gigante: reporte de...,ANTECEDENTES:Los ganglioneuromas son tumores e...,None,https://revistamexicanadeurologia.org.mx/index...,https://revistamexicanadeurologia.org.mx/index...
239,Revista Mexicana de Urología,III. Medicina y Ciencias de la Salud,Urología,Síndrome de Wunderlich en una paciente con esc...,"ANTECEDENTES:El síndrome de Wunderlich, o hemo...",None,https://revistamexicanadeurologia.org.mx/index...,https://revistamexicanadeurologia.org.mx/index...
240,Revista Mexicana de Urología,III. Medicina y Ciencias de la Salud,Urología,Litiasis renal bilateral y catéter doble J cal...,ANTECEDENTES:Debido al progreso en el equipo y...,None,https://revistamexicanadeurologia.org.mx/index...,https://revistamexicanadeurologia.org.mx/index...
241,Revista Mexicana de Urología,III. Medicina y Ciencias de la Salud,Urología,Colocación de un balón de oclusión ureteral pa...,ANTECEDENTES:La incidencia de fístula urinaria...,None,https://revistamexicanadeurologia.org.mx/index...,https://revistamexicanadeurologia.org.mx/index...


In [41]:
df_catalogo.to_csv('Revista093.01.csv', index=False)